## Extract ingredients and receipes

In [1]:
import os
import json
import numpy as np
import pandas as pd
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from tqdm import tqdm

In [2]:
data_dir = './data'
# first_dirs = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x)) and not x.startswith('.')]
# second_dirs = [os.listdir(os.path.join(data_dir, x)) for x in first_dirs]

In [3]:
def extract_json(d):
    first_dirs = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x)) and not x.startswith('.')]
    second_dirs = [os.listdir(os.path.join(data_dir, x)) for x in first_dirs]
    allfiles = [[os.path.join(data_dir, x, y) for y in ys] for x, ys in zip(first_dirs, second_dirs)]
#     del first_dirs
#     del second_dirs
    allfiles_flat = [val for sublist in allfiles for val in sublist]
#     del allfiles
    return allfiles_flat
   

def to_dataframe(file_list):
    df = pd.DataFrame()
    for thf in file_list:
        with open(thf, encoding='utf-8') as json_file:
            data = json.load(json_file).get('data').get('page').get('entities')
            for x in data:
                df = df.append(x, ignore_index=True)
                # print(df['id'])
    return df

In [4]:
my_df = to_dataframe(extract_json(data_dir))
# my_df.set_index('id')
# my_df

In [532]:
ingredients = list()
for x in my_df['ingredientKeywords'].str.split(','):
    ingredients.append([y.strip() for y in x])
#     ingredients.append(tuple(y.strip() for y in x))
# ingredients

In [6]:
len(ingredients)

3750

In [7]:
recipes = list()
for _,x in my_df['title'].iteritems():
    recipes.append(x.strip())
# recipes

In [8]:
len(recipes)

3750

In [476]:
_ingredients = {
    'id': [],
    'recipe': [],
    'ingredient': []
}
for i, row in my_df.iterrows():
    row_id = row['id']
    row_title = row['title'].strip()
    for ing in row['ingredientKeywords'].split(','):
        if ing.strip() == '':
#             print(row_title)
            continue
        _ingredients['id'].append(row_id)
        _ingredients['recipe'].append(row_title)
        _ingredients['ingredient'].append(ing.strip())
df_ingredient = pd.DataFrame(_ingredients)
del _ingredients

## Cleaning

In [483]:
def clean_ingre(x: str):
    x = x.strip('*').\
        strip('+').\
        strip('-').\
        strip('~').\
        strip(':').\
        strip('.').\
        strip('1.').\
        strip('2.').\
        strip('3.').\
        strip('4.').\
        strip('5.').\
        strip('6.').\
        strip('7.').\
        strip('8.').\
        strip('9.').\
        strip('10.').\
        strip(' ')
    if x == '###วัตถุดิบซอส###' or\
       x == '###วัตถุดิบสำหรับซุป###':
        return ''
    if x.startswith('A) ') or\
        x.startswith('B) ') or\
        x.startswith('C) ') or\
        x.startswith('D) ') or\
        x.startswith('E) ') or\
        x.startswith('A. ') or\
        x.startswith('B. '):
        return ''
    if 'กระดาษ' in x.lower():
        return ''
    if 'ส่วนผสม' in x.lower():
        return ''
    
    x = x.replace('\u200d','')
    x = x.replace('\u200b','')
    return x

In [484]:
def translate(x):
    
    if 'bacon' in x.lower():
        return 'เบคอน'
    if 'เบค่อน' in x:
        return 'เบคอน'
    if 'เบคอน' in x:
        return 'เบคอน'
    
    if 'มาม่า' in x:
        return 'บะหมี่กึ่งสำเร็จรูป'
    if 'บะหมี่กึ่ง' in x:
        return 'บะหมี่กึ่งสำเร็จรูป'
    
    if 'บะหมี่' in x and 'บะหมี่กึ่ง' not in x:
        return 'บะหมี่'
    
    if 'มัสตาร์ด' in x or 'mustard' in x.lower():
        return 'มัสตาร์ด'
    
    if 'ช็อค' in x or 'chocolate' in x.lower():
        return 'ช็อคโกแลต'
    
    if 'โกโก้' in x or 'cacao' in x.lower():
        return 'โกโก้'
    
    if 'โย' in x or 'yogurt' in x.lower():
        return 'โยเกิร์ต'
    
    if 'วานิล' in x or 'วนิล' in x or 'vanilla' in x.lower():
        return 'วานิลลา'
    
    if 'เนย' in x or 'butter' in x.lower():
        return 'เนย'
    
    if 'มาการีน' in x or 'margarine' in x.lower():
        return 'มาการีน'
    
    if 'กะทิ' in x:
        return 'เนย'
    
    if 'ซี่โครง' in x and 'ซี่โครงไก่' not in x:
        return 'ซี่โครงหมู'
    
    if 'คนอร์' in x:
        if 'หมู' in x:
            return 'คนอร์หมู'
        if 'ไก่' in x:
            return 'คนอร์ไก่'
        if 'ลาบ' in x:
            return 'คนอร์ลาบ'
        if 'เห็ด' in x:
            return 'คนอร์เห็ด'
        if 'ผัก' in x:
            return 'คนอร์ผัก'
        if 'ซอส' in x or 'ขวด':
            return 'คนอร์ซอส'
        if 'ก้อน' in x:
            return 'คนอร์ก้อน'
        if 'ผง' in x:
            return 'คนอร์ผง'
            
    
#     if 'คนอร์' in x
    
#     if 'ไอซ' in x or 'icing' in x.lower():
#         return 'ไอซิ่ง'
    
    return x

In [521]:
# df_ingredient = df_ingredient[~df_ingredient['ingredient'].isnull()]
# df_ingredient = df_ingredient[~df_ingredient['ingredient'].isna()]
df_ingredient.loc[:,'ingredient'] = df_ingredient.loc[:,'ingredient'].apply(clean_ingre)
#ผักชี before chesse
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ผักชี', case=False),'ingredient'] = 'ผักชี'
df_ingredient.loc[
    (
        (df_ingredient['ingredient'].str.contains('ชีส', case=False)) |
        (df_ingredient['ingredient'].str.contains('cheese', case=False))
    )
    , 'ingredient'] = 'ชีส'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กลิ่นกาแฟ', case=False),'ingredient'] = 'กลิ่นกาแฟ'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('espresso', case=False)) |
    (
        (df_ingredient['ingredient'].str.contains('กาแฟ', case=False)) &
        (~df_ingredient['ingredient'].str.contains('กลิ่นกาแฟ', case=False))
    )
    , 'ingredient'] = 'กาแฟ'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำปลา', case=False),'ingredient'] = 'น้ำปลา'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาหมึก', case=False),'ingredient'] = 'ปลาหมึก'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กล้วย', case=False),'ingredient'] = 'กล้วย'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('เม็ดมะม่วง', case=False),'ingredient'] = 'เม็ดมะม่วงหิมพานต์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เมล็ดมะม่วง', case=False),'ingredient'] = 'เม็ดมะม่วงหิมพานต์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ถั่วหิมพานต์', case=False),'ingredient'] = 'เม็ดมะม่วงหิมพานต์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('มะม่วงหิมพานต์', case=False),'ingredient'] = 'เม็ดมะม่วงหิมพานต์'
# df_ingredient.loc[df_ingredient['ingredient'].str.contains('เมล็ดมะม่วง', case=False),'ingredient'] = 'เม็ดมะม่วงหิมพานต์'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไอซ', case=False),'ingredient'] = 'ไอซิ่ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('icing', case=False),'ingredient'] = 'ไอซิ่ง'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำตาล', case=False),'ingredient'] = 'น้ำตาล'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('sugar', case=False),'ingredient'] = 'น้ำตาล'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กะป', case=False),'ingredient'] = 'กะปิ'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กะหล่ำ', case=False),'ingredient'] = 'กะหล่ำ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กระหล่ำ', case=False),'ingredient'] = 'กะหล่ำ'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กระชาย', case=False),'ingredient'] = 'กระชาย'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กะเจี๊ยบ', case=False),'ingredient'] = 'กระเจี๊ยบ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กระเจี๊ยบ', case=False),'ingredient'] = 'กระเจี๊ยบ'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('garlic', case=False),'ingredient'] = 'กระเทียม'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กะเทียม', case=False),'ingredient'] = 'กระเทียม'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กระเทียม', case=False),'ingredient'] = 'กระเทียม'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุนเชียง', case=False),'ingredient'] = 'กุนเชียง'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กระดูกหมู', case=False),'ingredient'] = 'กระดูกหมู'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('เต้าหู้ไข่', case=False),'ingredient'] = 'เต้าหู้ไข่'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่เค็ม', case=False),'ingredient'] = 'ไข่เค็ม'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่ขาว', case=False),'ingredient'] = 'ไข่ขาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่แดง', case=False),'ingredient'] = 'ไข่แดง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่ดาว', case=False),'ingredient'] = 'ไข่ดาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่เจียว', case=False),'ingredient'] = 'ไข่เจียว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่ไก่', case=False),'ingredient'] = 'ไข่ไก่'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไส้กรอก', case=False),'ingredient'] = 'ไส้กรอก'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('สามชั้น', case=False),'ingredient'] = 'หมูสามชั้น'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('3ชั้น', case=False),'ingredient'] = 'หมูสามชั้น'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูมามชั้น', case=False),'ingredient'] = 'หมูสามชั้น'

df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งฝอย', case=False),'ingredient'] = 'กุ้งฝอย'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไข่กุ้ง', case=False),'ingredient'] = 'ไข่กุ้ง'

# df_ingredient.loc[df_ingredient['ingredient'].str.contains('ขนมปัง', case=False),'ingredient'] = 'ขนมปัง'

df_ingredient.loc[:,'ingredient'] = df_ingredient.loc[:,'ingredient'].apply(clean_ingre).apply(translate)

df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูบด', case=False),'ingredient'] = 'หมูสับ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ข้าวตัง', case=False),'ingredient'] = 'ข้าวตัง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูหยอง', case=False),'ingredient'] = 'หมูหยอง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูหย็อง', case=False),'ingredient'] = 'หมูหยอง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูสับ', case=False),'ingredient'] = 'หมูสับ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เลือดหมู', case=False),'ingredient'] = 'เลือดหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ตับหมู', case=False),'ingredient'] = 'ตับหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('สันนอก', case=False),'ingredient'] = 'หมูสันนอก'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูสันใน', case=False),'ingredient'] = 'หมูสันใน'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('สันในหมู', case=False),'ingredient'] = 'หมูสันใน'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('หมู', case=False)) &
    (df_ingredient['ingredient'].str.contains('สันใน', case=False))
    ,'ingredient'] = 'หมูสันใน'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('สันในไก่', case=False),'ingredient'] = 'สันในไก่'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไก่บด', case=False),'ingredient'] = 'ไก่สับ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำสลัด', case=False),'ingredient'] = 'น้ำสลัด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ซีอิ้วดำ', case=False),'ingredient'] = 'ซีอิ้วดำ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ซีอิ้วหวาน', case=False),'ingredient'] = 'ซีอิ้วดำ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ซีอิ้วข', case=False),'ingredient'] = 'ซีอิ้วขาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ซีอิ๊วข', case=False),'ingredient'] = 'ซีอิ้วขาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ซีอิ้วปรุง', case=False),'ingredient'] = 'ซีอิ้วขาว'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('ซีอิ้ว', case=False)) &
    (df_ingredient['ingredient'] != 'ซีอิ้วดำ')
    ,'ingredient'] = 'ซีอิ้วขาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งแห้ง', case=False),'ingredient'] = 'กุ้งแห้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูยอ', case=False),'ingredient'] = 'หมูยอ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หนังหมู', case=False),'ingredient'] = 'หนังหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('แฮม', case=False),'ingredient'] = 'แฮม'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ทูน่า', case=False),'ingredient'] = 'ทูน่า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำมันพืช', case=False),'ingredient'] = 'น้ำมันพืช'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('สันคอหมู', case=False),'ingredient'] = 'สันคอหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูสันคอ', case=False),'ingredient'] = 'สันคอหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูสไล', case=False),'ingredient'] = 'หมูสไลด์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กากหมู', case=False),'ingredient'] = 'กากหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เนื้อหมู', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูติดมัน', case=False),'ingredient'] = 'หมูติดมัน'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('หมู', case=False)) &
    (df_ingredient['ingredient'].str.contains('ผง', case=False))
    ,'ingredient'] = 'ผงปรุงรสหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูเด้ง', case=False),'ingredient'] = 'หมูเด้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูบะช่อ', case=False),'ingredient'] = 'หมูเด้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูชิ้น', case=False),'ingredient'] = 'หมูชิ้น'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไส้หมู', case=False),'ingredient'] = 'ไส้หมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หัวหมู', case=False),'ingredient'] = 'หัวหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ลูกชิ้นหมู', case=False),'ingredient'] = 'ลูกชิ้นหมู'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('หมู', case=False)) &
    (df_ingredient['ingredient'].str.contains('ซุป', case=False))
    ,'ingredient'] = 'ซุปก้อนรสหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูหมัก', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูหั่น', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูนุ่ม', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เนื้ิอหมู', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำมันหอย', case=False),'ingredient'] = 'น้ำมันหอย'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ขาหมู', case=False),'ingredient'] = 'ขาหมู'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('หมู', case=False)) &
    (df_ingredient['ingredient'].str.contains('แดง', case=False))
    ,'ingredient'] = 'หมูแดง'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('มันหมู', case=False)) &
    (df_ingredient['ingredient'] != 'น้ำมันหมู')
    ,'ingredient'] = 'มันหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไส้หมู', case=False),'ingredient'] = 'ไส้หมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ใส้หมู', case=False),'ingredient'] = 'ไส้หมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('คอหมู', case=False),'ingredient'] = 'คอหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('โบโล', case=False),'ingredient'] = 'โบโลน่า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูแผ่น', case=False),'ingredient'] = 'หมูแผ่น'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูทอด', case=False),'ingredient'] = 'หมูทอด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูทอด', case=False),'ingredient'] = 'หมูทอด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูฝอ', case=False),'ingredient'] = 'หมูฝอย'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำเปล่า', case=False),'ingredient'] = 'น้ำเปล่า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมูชิ้น', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หมู ', case=False),'ingredient'] = 'เนื้อหมู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('แคร็กเกอร์', case=False),'ingredient'] = 'แครกเกอร์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('แครกเกอร์', case=False),'ingredient'] = 'แครกเกอร์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ลูกชิ้นปลา', case=False),'ingredient'] = 'ลูกชิ้นปลา'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('แซล', case=False),'ingredient'] = 'แซลม่อน'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาข้าวสาร', case=False),'ingredient'] = 'ปลาข้าวสาร'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('หอยลาย', case=False),'ingredient'] = 'หอยลาย'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำพริกเผา', case=False),'ingredient'] = 'น้ำพริกเผา'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาทู', case=False),'ingredient'] = 'ปลาทู'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาสลิด', case=False),'ingredient'] = 'ปลาสลิด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาดอลลี่', case=False),'ingredient'] = 'ปลาดอลลี่'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาดอร์รี่', case=False),'ingredient'] = 'ปลาดอลลี่'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาดอรี่', case=False),'ingredient'] = 'ปลาดอลลี่'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำสต็อ', case=False),'ingredient'] = 'น้ำสต็อก'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('าร้า', case=False),'ingredient'] = 'ปลาร้า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลานิ', case=False),'ingredient'] = 'ปลานิล'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('พริกไทยดำ', case=False),'ingredient'] = 'พริกไทดำ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('พริกไทดำ', case=False),'ingredient'] = 'พริกไทดำ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('พริกไทยขาว', case=False),'ingredient'] = 'พริกไทขาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('พริกไทขาว', case=False),'ingredient'] = 'พริกไทขาว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลากระป๋อง', case=False),'ingredient'] = 'ปลากระป๋อง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาซาบะ', case=False),'ingredient'] = 'ปลาซาบะ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาส้ม', case=False),'ingredient'] = 'ปลาส้ม'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาตะเพียน', case=False),'ingredient'] = 'ปลาตะเพียน'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลากรา', case=False),'ingredient'] = 'ปลากราย'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาสำลี', case=False),'ingredient'] = 'ปลาสำลี'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('จาระเม็ด', case=False),'ingredient'] = 'ปลาจาระเม็ด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('จรเม็ด', case=False),'ingredient'] = 'ปลาจาระเม็ด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาเค็ม', case=False),'ingredient'] = 'ปลาเค็ม'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาเก๋า', case=False),'ingredient'] = 'ปลาเก๋า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปลาช่อน', case=False),'ingredient'] = 'ปลาช่อน'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ถั่วลิสง', case=False),'ingredient'] = 'ถั่วลิสง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('almond', case=False),'ingredient'] = 'อัลมอนด์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('อัลมอนด์', case=False),'ingredient'] = 'อัลมอนด์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ปีกไก่', case=False),'ingredient'] = 'ปีกไก่'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เมล็ดฟักทอง', case=False),'ingredient'] = 'เมล็ดฟักทอง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เกลือ', case=False),'ingredient'] = 'เกลือ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('salt', case=False),'ingredient'] = 'เกลือ'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('น้ำ', case=False)) &
    (df_ingredient['ingredient'].str.contains('ปลา', case=False))
    ,'ingredient'] = 'น้ำปลา'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เคยกุ้ง', case=False),'ingredient'] = 'เคยกุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('คางกุ้ง', case=False),'ingredient'] = 'คางกุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ก้ามกุ้ง', case=False),'ingredient'] = 'ก้ามกุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ไช่กุ้ง', case=False),'ingredient'] = 'ไข่กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งสด', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งขาว', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งนาง', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งสับ', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งตัวเล็ก', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งชีแฮ้', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งชี้แฮ้', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งสด', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งลวก', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้ง ไซด์ใหญ่', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('🍤', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งแช่แข็ง', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เนื้อกุ้ง', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้ง ตัดหัว', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งแก้ว', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งเป็นตัว', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งขนาดเล็ก', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งสุก', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งทอด', case=False),'ingredient'] = 'กุ้งทอด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำพริกมันกุ้ง', case=False),'ingredient'] = 'น้ำพริกมันกุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งตัวๆ', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งแกะ', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งต้ม', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งสเ', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งแชบ๊วย', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งหรือ', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งปอกเปลือก', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งไซส์ใหญ่', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้ง 3 ตัว', case=False),'ingredient'] = 'กุ้ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('กุ้งลายเสือ', case=False),'ingredient'] = 'กุ้งลายเสือ'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('ล็อบสเตอร์', case=False),'ingredient'] = 'ล็อบสเตอร์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('lobster', case=False),'ingredient'] = 'ล็อบสเตอร์'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('นมจืด', case=False),'ingredient'] = 'นมจืด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำแข็ง', case=False),'ingredient'] = 'น้ำแข็ง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('นมข้นหวาน', case=False),'ingredient'] = 'นมข้นหวาน'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำมันมะพร้าว', case=False),'ingredient'] = 'น้ำมันมะพร้าว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('นมข้นจืด', case=False),'ingredient'] = 'นมข้นจืด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เส้นพาสต้า', case=False),'ingredient'] = 'เส้นพาสต้า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('pasta', case=False),'ingredient'] = 'เส้นพาสต้า'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('น้ำมันมะกอก', case=False),'ingredient'] = 'น้ำมันมะกอก'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('มายองเนส', case=False),'ingredient'] = 'มายองเนส'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('เกล็ดขนมปัง', case=False),'ingredient'] = 'เกล็ดขนมปัง'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('แป้งขนมปัง', case=False),'ingredient'] = 'แป้งขนมปัง'
df_ingredient.loc[
    (df_ingredient['ingredient'].str.contains('ขนมปัง', case=False)) &
    (df_ingredient['ingredient'] != 'เกล็ดขนมปัง') &
    (df_ingredient['ingredient'] != 'แป้งขนมปัง')
    ,'ingredient'] = 'ขนมปัง'
df_ingredient.loc[df_ingredient['ingredient'].str.startswith('นมเปรี้ยว','ingredient')] = 'นมเปรี้ยว'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('อโวคาโด', case=False),'ingredient'] = 'อโวคาโด'
df_ingredient.loc[df_ingredient['ingredient'].str.contains('Avocado', case=False),'ingredient'] = 'อโวคาโด'
# df_ingredient.loc[
#     (df_ingredient['ingredient'].str.contains('bacon', case=False)) |
#     (df_ingredient['ingredient'].str.contains('เบค่อน', case=False)) |
#     (df_ingredient['ingredient'].str.contains('เบคอน', case=False))
#     , 'ingredient'] = 'เบคอน'
# df_ingredient.loc[
#     (df_ingredient['ingredient'].str.contains('บะหมี่กึ่ง', case=False))
#     , 'ingredient'] = 'บะหมี่กึ่งสำเร็จรูป'
# df_ingredient.loc[
#     (
#         (df_ingredient['ingredient'].str.contains('บะหมี่', case=False)) &
#         (~df_ingredient['ingredient'].str.contains('บะหมี่กึ่ง', case=False))
#     )
#     , 'ingredient'] = 'บะหมี่'



df_ingredient = df_ingredient[df_ingredient['ingredient'] != '']
df_ingredient = df_ingredient[~df_ingredient['ingredient'].isnull()]
# df_ingredient = df_ingredient[~df_ingredient['ingredient'].isna()]

In [522]:
df_ingredient[df_ingredient['ingredient'].str.contains('นม', case=False)]['ingredient'].unique()
df_ingredient[df_ingredient['ingredient'].str.contains('salt', case=True)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('pasta', case=True)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('pasta', case=True)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('สะเต๊ะ', case=False)]['ingredient'].unique()
# df_ingredient.loc[
#     (df_ingredient['ingredient'].str.contains('หมู', case=False)) &
#     (df_ingredient['ingredient'].str.contains('ผง', case=False))
#     ,'ingredient']
# df_ingredient.loc[
#     (df_ingredient['ingredient'].str.contains('น้ำ', case=False)) &
#     (df_ingredient['ingredient'].str.contains('ปลา', case=False))
#     ,'ingredient']
# df_ingredient[df_ingredient['ingredient'].str.contains('ลูกชิ้นปลา', case=False)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('ลูกชิ้น', case=False)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('ปลา', case=False)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('เกลือ', case=False)]['ingredient'].unique()
# df_ingredient[df_ingredient['ingredient'].str.contains('เมล็ดฟักทอง', case=False)]['ingredient'].unique()


array([], dtype=object)

In [524]:
# sorted(df_ingredient['ingredient'].unique())

In [ ]:
# with open('./unique_ingredients.txt', 'w', encoding='utf-8') as f:
#     for x in sorted(unique_ingredients):
#         f.write(x)
#         f.write('\n')

In [ ]:
# all_recipes_raw_list = list(all_recipes_raw)

# chunked_list = list(chunks(all_recipes_raw_list, 400))
# random.shuffle(chunked_list)

In [ ]:
# find_n_ingredients = 6000
# select_top_n_ingredients = 3500
# ingredient_preselection = list()

# p = Pool(16)

# for r_chunk in tqdm(chunked_list):
#     r_chunk_new = p.map(clean_ingredient_list, r_chunk)
#     for r_new in r_chunk_new:
#         ingredient_preselection.extend(r_new)
        
#     unique_ingredient_count = len(set(ingredient_preselection))
#     if unique_ingredient_count > find_n_ingredients:
#         break
#     print(f'Found {unique_ingredient_count} unique ingredients') 
        
# c = Counter(ingredient_preselection)
# clean_ingredients = list([i for i, n in c.most_common()][:select_top_n_ingredients])
# assert len(clean_ingredients) == select_top_n_ingredients

In [615]:
df_ingredient.to_csv('df_ingredient.csv', encoding='utf-8')

## Preparing

In [535]:
unique_ingredients = list(set(df_ingredient['ingredient'].unique()))

In [536]:
df_ingredient.head()

,id,recipe,ingredient
0,7980ed28ce35467482b9858b86a5650d,ยำคะน้าหมูสับ,ผักคะน้า
1,7980ed28ce35467482b9858b86a5650d,ยำคะน้าหมูสับ,หมูสับ
2,7980ed28ce35467482b9858b86a5650d,ยำคะน้าหมูสับ,คนอร์หมู
3,7980ed28ce35467482b9858b86a5650d,ยำคะน้าหมูสับ,น้ำมะนาว
4,7980ed28ce35467482b9858b86a5650d,ยำคะน้าหมูสับ,หอมแดง


In [551]:
# df_ingredient['indexed']
# for r in df_ingredient.iterrow():
ingredients = list()
for k, g in df_ingredient.groupby('recipe'):
    igr = list()
    for i, r in g.iterrows():
#         print(r)
        igr.append(r['ingredient'])
    ingredients.append(igr)
# for x in df_ingredient['ingredient']:
#     ingredients.append([y.strip() for y in x])

In [614]:
with open('./ingredients.txt', 'w', encoding='utf-8') as f:
    for x in sorted(ingredients):
        f.write(','.join(x))
        f.write('\n')

In [ ]:
# clean_ingredients = [y.strip() for x in ingredients for y in x]
# clean_ingredients

In [ ]:
# sorted(clean_ingredients)

In [ ]:
# len(clean_ingredients)

In [552]:
indexed_recipes = []

for r in tqdm(ingredients):
    if r:
#         indexed_recipes.append(np.array([clean_ingredients.index(i) for i in r]))
        indexed_recipes.append(np.array([unique_ingredients.index(i) for i in r]))
    
indexed_recipes = np.array(indexed_recipes)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3594/3594 [00:01<00:00, 2632.97it/s]


In [553]:
len(indexed_recipes)

3594

In [554]:
indexed_recipes

array([array([ 448, 4055, 1688,   34,  481, 2057, 3500, 1451, 2714, 4078]),
       array([ 302,  481, 3164,  669,  345, 3253, 3290]),
       array([ 302,  337, 4151, 3500,  345, 2770, 1420]), ...,
       array([2715, 3907,  584,   86]),
       array([4078, 2808, 4182, 4145, 3142, 2377, 2770, 3253, 3068,  252, 2738,
       1935, 3729, 3799,  731, 2377, 3277, 2940]),
       array([4162, 2596, 1891, 2808, 1092,  942, 1800, 1236,   48])],
      dtype=object)

In [555]:
# maybe it's a good time to save?
# np.savez_compressed('wongnet', ingredients=clean_ingredients, recipes=indexed_recipes)
np.savez_compressed('wongnet', ingredients=unique_ingredients, recipes=indexed_recipes)

In [556]:
class Recipes:

    def __init__(self):
        pass
#         self.load_recipes(data_path)

    def load_recipes(self, data_path):
#         with np.load('simplified-recipes-1M.npz', allow_pickle=True) as data:
        with np.load(data_path, allow_pickle=True) as data:
            self.recipes = data['recipes']
            self.ingredients = data['ingredients']
        np.random.shuffle(self.recipes)

        self.train_size = int(0.97 * len(self.recipes))
        self.test_size = len(self.recipes) - self.train_size
        self.train_recipes = self.recipes[:self.train_size]
        self.test_recipes = self.recipes[self.train_size:]
        print(self.train_recipes.shape, self.test_recipes.shape)

    def onehot_to_hr_recipe(self, ind):
        r = []
        for i, v in enumerate(ind):
            if v == 1:
                r.append(self.ingredients[i])
        return len(r), r

    """def get_batch_alt(recipes_array, batch_size, set_n_to_zero=1):
        while True:
            print(len(recipes_array), 'dataset iterator epoch start')
            offset = 0
            for batch_end_offset in range(batch_size, len(recipes_array)+1, batch_size):

                batch_data = recipes_array[batch_end_offset-batch_size:batch_end_offset]

                batch_y = np.zeros((batch_size, len(ingredients)))
                for i, row in enumerate(batch_data):
                    try:
                        batch_y[i, row] = 1
                    except IndexError: pass

                batch_x = np.copy(batch_y)
                for i, row in enumerate(batch_data):
                    try:
                        ind = np.random.choice(row, set_n_to_zero, replace=False)
                        batch_x[i, ind] = 0
                    except (IndexError, ValueError): pass

                #indices = np.random.random((batch_size, len(ingredients))) > 0.1
                #batch_x[indices] = 0
                yield batch_x, batch_y"""

    def labels_to_batch(self, l):
        l = self.filter_ingr(l)
        assert len(l) > 2
        k = [self.hr_to_indices_list(l)]

        x, y = self.batch_iterator(k, 1).__next__()
        return x + y

    def hr_to_indices_list(self, l):
        r = np.zeros(len(l), dtype=np.int32)
        for c, i in enumerate(l):
            r[c] = np.where(self.ingredients == i)[0][0]
        return r

    def batch_iterator(self, recipes_array, batch_size):
        while True:
            # print('\n', len(recipes_array), 'dataset iterator epoch start')
            offset = 0
            for batch_end_offset in range(batch_size, len(recipes_array) + 1, batch_size):
                batch_data = recipes_array[batch_end_offset - batch_size:batch_end_offset]

                batch_y = np.zeros((batch_size, len(self.ingredients)))
                batch_x = np.zeros((batch_size, len(self.ingredients)))

                for i, row in enumerate(batch_data):
                    remove_max_n_elements = 5
                    remove_max_one_nth_of_elements = 4
                    try:
                        n_to_remove = np.random.randint(1, 1 + max(0, min(remove_max_n_elements,
                                                                          math.floor(len(row) / remove_max_one_nth_of_elements))))

                        ind = np.random.randint(0, len(row), n_to_remove)
                        row_with_ingr_removed = np.delete(row, ind) # remove some ingredients
                        batch_y[i, row[ind]] = 1
                        batch_x[i, row_with_ingr_removed] = 1
                    except (IndexError, ValueError):
                        pass

                yield batch_x, batch_y

    def get_batch_count(self, recipes_array, bs):
        return len(list(range(bs, len(recipes_array), bs)))

    def filter_ingr(self, l):
        return [k for k in l if k in self.ingredients or print(k, 'not found')]

    def match_ingr(self, i):
        l = []
        for u in self.ingredients:
            if i in u or u in i:
                l.append(u)
        return l

In [596]:
# row = r.train_recipes
# batch_y = np.zeros((len(r.recipes), len(r.ingredients)))
# batch_x = np.zeros((len(r.recipes), len(r.ingredients)))
# remove_max_n_elements = 5
# remove_max_one_nth_of_elements = 4

# n_to_remove = np.random.randint(1, 1 + max(0, min(remove_max_n_elements,
#                                                   math.floor(len(row) / remove_max_one_nth_of_elements))))

# ind = np.random.randint(0, len(row), n_to_remove)
# row_with_ingr_removed = np.delete(row, ind)
# row_with_ingr_removed
# batch_y[i, row[ind]] = 1
# batch_x[i, row_with_ingr_removed] = 1

# Neural Network

In [557]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import math
import datetime
import os
# from util import Recipes
import random
print(tf.__version__)

2.0.0


In [558]:
r = Recipes()
r.load_recipes('wongnet.npz')

(3486,) (108,)


In [559]:
g = r.batch_iterator(r.train_recipes, 200)
x, y = g.__next__()
i = random.randint(0, 199)
print(r.onehot_to_hr_recipe(x[i]))
print()
print(r.onehot_to_hr_recipe(y[i]))

(8, ['น้ำมันพืช', 'แครอทหั่นเต๋า', 'ข้าวสวย(แช่ตู้เย็นค้างคืน)', 'เบคอน', 'ซอสปรุงรส', 'พริกไทยป่น', 'น้ำตาล', 'ต้นหอมซอย'])

(2, ['เนย', 'ไข่ไก่'])


In [588]:
# batch_size = 256
batch_size = 54
# valid_batch_size = 512
valid_batch_size = 107
epochs = 42

# model = tf.keras.models.Sequential([tf.keras.layers.Dense(500, input_shape=(3500,)),
# model = tf.keras.models.Sequential([tf.keras.layers.Dense(500, input_shape=(6408,)),
model = tf.keras.models.Sequential([tf.keras.layers.Dense(500, input_shape=(4230,)),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(330),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(300),
                                    tf.keras.layers.LeakyReLU(),
                                                                                                            
                                    tf.keras.layers.Dense(330),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(500),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(len(r.ingredients), activation='sigmoid')
                                   ])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, update_freq=batch_size*64)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 500)               2115500   
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 500)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 330)               165330    
_________________________________________________________________
leaky_re_lu_41 (LeakyReLU)   (None, 330)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 300)               99300     
_________________________________________________________________
leaky_re_lu_42 (LeakyReLU)   (None, 300)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 330)              

In [589]:
history = model.fit_generator(r.batch_iterator(r.train_recipes, batch_size=batch_size), 
                              epochs=epochs, 
                              steps_per_epoch=r.get_batch_count(r.train_recipes, batch_size), 
                              validation_data=r.batch_iterator(r.test_recipes, batch_size=valid_batch_size), 
                              validation_steps=r.get_batch_count(r.test_recipes, valid_batch_size),
                              #callbacks=[tensorboard_callback]
                             )

Epoch 1/42
64/64 [==============================] - 2s 34ms/step - loss: 9.2667 - categorical_accuracy: 0.0417 - val_loss: 8.3335 - val_categorical_accuracy: 0.0561
Epoch 2/42
64/64 [==============================] - 2s 32ms/step - loss: 8.3977 - categorical_accuracy: 0.0411 - val_loss: 7.7595 - val_categorical_accuracy: 0.0280
Epoch 3/42
64/64 [==============================] - 2s 32ms/step - loss: 8.2435 - categorical_accuracy: 0.0359 - val_loss: 8.4791 - val_categorical_accuracy: 0.0467
Epoch 4/42
64/64 [==============================] - 2s 33ms/step - loss: 8.0415 - categorical_accuracy: 0.0448 - val_loss: 9.0238 - val_categorical_accuracy: 0.0280
Epoch 5/42
64/64 [==============================] - 2s 32ms/step - loss: 8.0003 - categorical_accuracy: 0.0469 - val_loss: 7.9478 - val_categorical_accuracy: 0.0654
Epoch 6/42
64/64 [==============================] - 2s 33ms/step - loss: 7.8135 - categorical_accuracy: 0.0428 - val_loss: 8.5818 - val_categorical_accuracy: 0.0467
Epoch 7/42

In [564]:
# r.match_ingr('น้ำตาลทราย')

['น้ำ', 'น้ำตาล']

In [567]:
def no_overlap(item, s):
    for k in s:
        if item in k or k in item:
            return False
    return True

In [599]:
# i = r.labels_to_batch(['หมูสับ', 'ไข่ไก่', 'เกลือ'])
i = r.labels_to_batch(['น้ำมัน', 'ผงปรุงรส', 'คะน้าฮ่องกง'])
p = model.predict_on_batch(i)
_, hr = r.onehot_to_hr_recipe(i[0]) 

print(hr, '-->\n')

n = 16
added_ingr = list(reversed(r.ingredients[np.argsort(p[0])[-n:]]))
added_ingr_p = list(reversed(p[0].numpy()[np.argsort(p[0])[-n:]]))

n_c = 0
for ingr, p in zip(added_ingr, added_ingr_p):
    no_overlap_ = no_overlap(ingr, set(hr))
    print(('' if no_overlap_ else '(')+ingr+('' if no_overlap_ else ')'), '|', str(p)[:5], '|', '.')
    #if no_overlap_: n_c += 1
    #if n_c > 6: break

[] -->

น้ำผึ้ง | 0.006 | .
น้ำแข็ง | 0.001 | .
นมข้น | 0.001 | .
ขนมปัง | 0.000 | .
เบคอน | 0.000 | .
นมสด | 0.000 | .
ซอสสไปซี่บลูโกกิ (ทำซอส) | 0.000 | .
กีวี่ | 9.554 | .
นมข้นจืด | 8.007 | .
นมข้นหวาน | 7.742 | .
น้ำเชื่อม | 7.641 | .
น้ำดื่ม | 6.920 | .
ใบเตย | 6.911 | .
งาดำ | 6.672 | .
น้ำมันทอด | 6.645 | .
กล้วย | 5.519 | .


In [625]:
g = r.batch_iterator(r.train_recipes, 1)
x, y = g.__next__()
# print(x)
# print(y)
# i = random.randint(0, 199)
print(r.onehot_to_hr_recipe(x[0]))
# # print()
print(r.onehot_to_hr_recipe(y[0]))

(10, ['M&M', 'ผงฟู', 'เนย', 'ช็อคโกแลต', 'ไข่ไก่', 'วานิลลา', 'น้ำตาล', 'แป้งตราว่าว', 'โกโก้', 'โอริโอ้'])
(1, ['เม็ดมะม่วงหิมพานต์'])


In [626]:
p = model.predict_on_batch(x)
# p

In [627]:
_, hr = r.onehot_to_hr_recipe(x[0])

In [628]:
n = 16
added_ingr = list(reversed(r.ingredients[np.argsort(p[0])[-n:]]))
added_ingr_p = list(reversed(p[0].numpy()[np.argsort(p[0])[-n:]]))

n_c = 0
for ingr, p in zip(added_ingr, added_ingr_p):
    no_overlap_ = no_overlap(ingr, set(hr))
    print(('' if no_overlap_ else '(')+ingr+('' if no_overlap_ else ')'), '|', str(p)[:5], '|', '.')

แป้งเค้ก | 0.700 | .
(แป้งตราว่าว) | 0.610 | .
(วานิลลา) | 0.483 | .
นมจืด | 0.437 | .
แป้งกวนไส้ | 0.364 | .
(น้ำตาล) | 0.359 | .
(โกโก้) | 0.310 | .
นดสดจืด | 0.301 | .
เม็ดมะม่วงหิมพานต์ | 0.260 | .
กล้วย | 0.248 | .
(ผงฟู) | 0.214 | .
(ไข่ไก่) | 0.210 | .
😍ส่วนที่ | 0.176 | .
(เนย) | 0.162 | .
นมข้นจืด | 0.149 | .
โอวาเลท | 0.125 | .
